<a href="https://colab.research.google.com/github/Suryansh1089/GroupProject/blob/main/NLPproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:
!pip install jsonlines
import jsonlines
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size =2500


In [5]:
import pandas as pd


In [6]:
type('twitter_train.jsonl')

str

In [8]:
twitter_train=pd.read_json('twitter_train.jsonl',lines='True')

In [9]:
twitter_train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_label=le.fit_transform(twitter_train['label'])
twitter_train['label']=train_label
twitter_train.head()

,label,response,context
0,1,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,1,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,1,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,1,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,1,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [12]:
type(twitter_train)

pandas.core.frame.DataFrame

In [13]:
twitter_train.shape

(5000, 3)

In [14]:

from collections import Counter
Counter(twitter_train['label'])


Counter({0: 2500, 1: 2500})

In [15]:
twitter_train['response'][0]

"@USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her .."

In [16]:
sentences=[]
cont=[]
labels=[]
for i in range(len(twitter_train)):
  sentences.append(twitter_train['response'][i])
  cont.append(twitter_train['context'][i])
  labels.append(twitter_train['label'][i])

In [17]:
len(sentences)

5000

In [18]:
sentences[0]

"@USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her .."

In [19]:

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]
training_cont=cont[0:training_size]
testing_cont=cont[training_size:]

In [20]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
tokenizer.fit_on_texts(training_cont)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
train_cont_sequences=tokenizer.texts_to_sequences(training_cont)
training_cont_padded = pad_sequences(train_cont_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_cont_sequences=tokenizer.texts_to_sequences(testing_cont)
test_cont_padded = pad_sequences(test_cont_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [21]:
import numpy as np
training_padded = np.array(training_padded)

training_cont_padded=np.array(training_cont_padded)
training_labels = np.array(training_labels)
testing_labels=np.array(testing_labels)
testing_padded=np.array(testing_padded)
test_cont_padded=np.array(test_cont_padded)

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=5000),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5000, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [24]:
arr=[1,2]

In [25]:
arr

[1, 2]

In [26]:
a=2
b=3
arr2=[a,b]

In [27]:
arr3=[arr,arr2]

In [28]:
arr3

[[1, 2], [2, 3]]

In [29]:
type(training_padded)

numpy.ndarray

In [30]:
num_epochs = 10
a=np.array([training_padded,training_cont_padded])
b=np.array([testing_padded,test_cont_padded])

In [31]:
type(a)

numpy.ndarray

In [32]:
type(a)
type(b)
len(a)
print(len(training_cont_padded))
print(len(training_padded))
print(len(test_cont_padded))
print(len(testing_padded))
print(len(training_labels))
print(len(testing_labels))


2500
2500
2500
2500
2500
2500


In [33]:
num_epoch=30
history=model.fit(training_padded,training_labels,num_epoch,validation_data=(testing_padded,testing_labels),verbose=2)

84/84 - 1s - loss: 0.5225 - accuracy: 1.0000 - val_loss: 1.2832 - val_accuracy: 0.0000e+00


In [34]:
sentence = ["Well now thats problematic AF ", "My 5 year old ... asked me why they are making fun of Native Americans ..", "i will take shit that didn't happen for  100", " no .. he actually in the gifted program and reads on second grade level .  ... and he knows Kansas City is in Missouri"]
context=["doesnt even know easy sum","doesnt even know easy sum"]
sequences = tokenizer.texts_to_sequences(sentence)
cont_sequences=tokenizer.texts_to_sequences(context)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
padded_cont=pad_sequences(cont_sequences,max_length,padding=padding_type,truncating='post')
print(model.predict(padded))

[[0.7386861 ]
 [0.7320602 ]
 [0.7368456 ]
 [0.72673786]]


In [35]:
import pandas as pd
twitter_test=pd.read_json('twitter_test.jsonl',lines='True')
twitter_test.head()

,context,response,id
0,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi...",twitter_1
1,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...,twitter_2
2,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...,twitter_3
3,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...,twitter_4
4,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...,twitter_5


In [36]:
twitter_test=twitter_test.drop(['id'],axis='columns')

In [37]:
twitter_test.head()

,context,response
0,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi..."
1,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...
2,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...
3,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...
4,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...


In [43]:
sentences=[]
for i in range(len(twitter_test)):
  sentences.append(twitter_test['response'][i])

In [45]:
sentences[0]

'@USER @USER @USER My 3 year old , that just finished reading Nietzsche and then asked me : " ayo papa why these people always trying to cancel someone on Twitter , trying to pretend like that makes them better themselves ? " . To which I replied " idk " , and he just " cuz hoes mad " . Im so proud . <URL>'

In [46]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index

result_sequences = tokenizer.texts_to_sequences(sentences)
result_padded = pad_sequences(result_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [47]:
import numpy as np
result_padded = np.array(result_padded)


In [48]:
model.predict(result_padded)

array([[0.7034223 ],
       [0.7289304 ],
       [0.7303229 ],
       ...,
       [0.73862684],
       [0.7364707 ],
       [0.72673213]], dtype=float32)

In [52]:
for i in range(len(result_padded)):
  print(model.predict(result_padded[i]))

Streaming output truncated to the last 5000 lines.
[[0.7223662 ]
 [0.7223662 ]
 [0.70399046]
 [0.70370317]
 [0.6386577 ]
 [0.6063657 ]
 [0.69386494]
 [0.71060324]
 [0.5873785 ]
 [0.5903241 ]
 [0.6787778 ]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.74401397]
 [0.7440139

FINAL PROBABILITIES OF STATEMENT BEING SARCASTIC
